# Budget Constraint

Bidding is limited to our campaign budget of 25000. 

In [1]:
import numpy as np
import xgboost
import pandas as pd

df = pd.read_csv("dataset/train.csv")
df_valid = pd.read_csv("dataset/validation.csv")
df_test = pd.read_csv("dataset/test.csv")

In [33]:
df_test.columns

Index(['weekday', 'hour', 'bidid', 'logtype', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [2]:
train_split = np.array_split(df, 2)
train = train_split[0]
train_y = train['bidprice']

cols = ['click', 'weekday', 'hour', 'bidid', 'logtype', 'userid', 'useragent',
       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
       'slotprice', 'creative', 'payprice', 'keypage',
       'advertiser', 'usertag']

train_x = train[cols]

## Click Prediction

In [25]:
features = ['weekday','hour', 'city', 'slotprice','slotheight', 'slotwidth', 'advertiser']
X = train[features]
y = train['bidprice']

model = xgboost.XGBClassifier()
model.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [28]:
test_split = np.array_split(df_valid, 1)
test = test_split[0]
test_x = test[features]
bidprice_predictions = model.predict(test_x)
test_x['bidprice_pred'] = bidprice_predictions
test_x['bidprice'] = test['bidprice']
test_x['abs_diff'] = abs(test_x['bidprice_pred'] - test_x['bidprice'])
test_x['abs_diff'].mean()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



0.045231176751215185

## Payprice Prediction

In [ ]:
features = ['click','weekday', 'hour', 'city', 'slotprice','slotheight', 'slotwidth', 'bidprice', 'advertiser']
X = train[features]
y = train['payprice']

model = xgboost.XGBClassifier()
model.fit(X, y)

In [5]:
features = ['click','weekday', 'hour', 'region', 'city', 'slotprice', 'slotheight', 'slotwidth','payprice', 'advertiser']
X = train[features]
y = train_y

model = xgboost.XGBClassifier()
model.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [41]:
test_split = np.array_split(df_test, 1)
test = test_split[0]
test_x = test[features]
test_x
test_predictions = model.predict(test_x)
test_x['bid_prediction'] = test_predictions
test_x['bidid'] = test['bidid']
test_x[['bidid','bid_prediction']].to_csv(path_or_buf='out.csv', header=['bidid','bidprice'], index=False)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
test_x['bid_prediction'].mean()

272.49757964163348

In [44]:
test_x.groupby('advertiser')['bid_prediction'].mean()

advertiser
1458    300.000000
2259    288.400528
2261    288.013038
2821    290.370220
2997    277.000000
3358    232.763999
3386    300.000000
3427    236.230712
3476    247.768108
Name: bid_prediction, dtype: float64